In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

import json
import requests

import numpy as np
import random
#from config import weather_api_key
#from config import g_key
#from citipy import citipy 

import gmaps
import os
series = pd.Series

In [2]:
# Load in file
streaming_file = "MoviesOnStreamingPlatforms_updated.csv"

In [3]:
# Read and display the CSV with Pandas
streaming_file_df = pd.read_csv(streaming_file)
streaming_file_df.head()
#streaming_file_df(16744 rows × 17 columns)

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [4]:
#dropna
streaming_file_df1=streaming_file_df.dropna(how='all')
streaming_file_df1.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [5]:
#remove % signs to allow for sorting and binning etc
streaming_file_df1['Rotten Tomatoes'] = streaming_file_df1['Rotten Tomatoes'].str.rstrip('%').astype('float') / 100.0

In [6]:
#remove strings that will prevent binning
streaming_file_df1['Age'] = streaming_file_df1['Age'].str.rstrip('+').astype('str')

In [7]:
streaming_file_df1.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13,8.8,0.87,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18,8.7,0.87,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13,8.5,0.84,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7,8.5,0.96,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18,8.8,0.97,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [8]:
#drop duplicates
streaming_file_df1.drop_duplicates(inplace=True)


In [9]:
#drop rows containing missing values in genres
Streaming_drop_df=streaming_file_df1.dropna(subset=['Genres','Age']) # check; 16469 rows × 17 columns
Streaming_drop_df.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13,8.8,0.87,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18,8.7,0.87,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13,8.5,0.84,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7,8.5,0.96,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18,8.8,0.97,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [10]:
df_NetflixGenre=Streaming_drop_df[Streaming_drop_df['Netflix']==1]
len(df_NetflixGenre)

3469

In [11]:
df_NetflixGenre.Genres.mode()

0    Comedy
dtype: object

In [12]:
Netflix_Genre=Streaming_drop_df[Streaming_drop_df['Netflix']==1].groupby('Genres')
Netflix_Genre.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13,8.8,0.87,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18,8.7,0.87,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13,8.5,0.84,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7,8.5,0.96,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18,8.8,0.97,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3515,3515,3516,Rehmataan,2012,nan,NaN,NaN,1,0,0,0,0,Rajveer Singh Maan,"Drama,Family,Romance",India,Punjabi,NaN
3521,3521,3522,Little Baby Bum: Go Buster,2019,nan,NaN,NaN,1,0,0,0,0,"Richard Hickey,Nick Hatton Jones","Animation,Family,Musical",NaN,NaN,NaN
3526,3526,3527,Los Niños Héroes de Chapultepec,2017,nan,NaN,NaN,1,0,0,0,0,Ignacio López Escrivá,"Documentary,History,War",Mexico,Spanish,87.0
3530,3530,3531,Oddbods: The Festive Menace,2017,nan,NaN,NaN,1,0,0,0,0,NaN,"Animation,Comedy,Family",NaN,NaN,NaN


In [13]:
df_NetflixGenre=Streaming_drop_df[Streaming_drop_df['Netflix']==1]
df_NetflixGenre

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13,8.8,0.87,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18,8.7,0.87,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13,8.5,0.84,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7,8.5,0.96,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18,8.8,0.97,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3546,3546,3547,Krish Trish and Baltiboy: Part II,2010,nan,NaN,NaN,1,0,0,0,0,Tilak Shetty,Animation,India,"Hindi,English",65.0
3549,3549,3550,Oddbods: Party Monsters,2018,nan,NaN,NaN,1,1,0,0,0,NaN,Animation,NaN,NaN,NaN
3555,3555,3556,Krish Trish and Baltiboy I,2009,nan,NaN,NaN,1,0,0,0,0,Tilak Shetty,Animation,India,"Hindi,English",65.0
3558,3558,3559,Shikari,1991,nan,NaN,NaN,1,0,0,0,0,"Joydip Mukherjee,Zakir Hossain","Action,Crime,Drama,Romance","Bangladesh,India",Bengali,145.0


In [18]:
df_HuluGenre=Streaming_drop_df[Streaming_drop_df['Hulu']==1]
len(df_HuluGenre)

897

In [15]:
##df_HuluGenre1=Streaming_drop_df[[Streaming_drop_df['Hulu']==1],Streaming_drop_df['Genres']]
#df_HuluGenre1

In [21]:
df_HuluGenre.Genres.count()

897

In [ ]:
#Streaming_drop2=df.dropna(subset, inplace=True)
             # .fillna(0)
             # .astype(int) # check; 16469 rows × 17 columns
#Streaming_drop2_df.head()


In [25]:
#keep columns needed for analysis
strm=Streaming_drop_df[['Title','Year','Age','IMDb','Rotten Tomatoes','Netflix','Hulu','Prime Video','Disney+','Directors',
                        'Genres','Country','Runtime']]
strm.head()

,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Directors,Genres,Country,Runtime
0,Inception,2010,13,8.8,0.87,1,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom",148.0
1,The Matrix,1999,18,8.7,0.87,1,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,136.0
2,Avengers: Infinity War,2018,13,8.5,0.84,1,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,149.0
3,Back to the Future,1985,7,8.5,0.96,1,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,116.0
4,"The Good, the Bad and the Ugly",1966,18,8.8,0.97,1,0,1,0,Sergio Leone,Western,"Italy,Spain,West Germany",161.0


In [26]:
#split genres and create new entry for each genre per movie
s=strm['Genres'].str.split(',').apply(series, 1).stack()
s.index=s.index.droplevel(-1)
s.name='Genres'
del strm['Genres']
strm_split_Genres=strm.join(s)

In [27]:
strm_split_Genres.head()


,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Directors,Country,Runtime,Genres
0,Inception,2010,13,8.8,0.87,1,0,0,0,Christopher Nolan,"United States,United Kingdom",148.0,Action
0,Inception,2010,13,8.8,0.87,1,0,0,0,Christopher Nolan,"United States,United Kingdom",148.0,Adventure
0,Inception,2010,13,8.8,0.87,1,0,0,0,Christopher Nolan,"United States,United Kingdom",148.0,Sci-Fi
0,Inception,2010,13,8.8,0.87,1,0,0,0,Christopher Nolan,"United States,United Kingdom",148.0,Thriller
1,The Matrix,1999,18,8.7,0.87,1,0,0,0,"Lana Wachowski,Lilly Wachowski",United States,136.0,Action


In [28]:
strm_split_Genres.Genres.count()

39098

In [ ]:
df_NetflixGenre=strm_split_Genres[strm_split_Genres['Netflix']==1]
len(df_NetflixGenre)

In [ ]:
#df_NetflixGenre=df_NetflixGenre.dropna(how='all')
#df_NetflixGenre.fillna(0, inplace=True, downcast='infer')
df_NetflixGenre.head()

In [ ]:
len(df_NetflixGenre['Genres'])

In [ ]:
len(df_NetflixGenre['IMDb'])

In [ ]:
# The next example will compute the Pearson correlation coefficient between "Genres" and "IMDb" for Netflix streaming
#df_NetflixGenre.fillna(0, inplace=True, downcast='infer')
df_NetflixGenre['Genres']=df_NetflixGenre['Genres'].astype('category')
df_NetflixGenre['IMDb']=df_NetflixGenre['IMDb'].astype('category')
#.cat.codes
df_NetflixGenre.fillna(0, inplace=True, downcast='infer')
df_NetflixGenre.plot.scatter('Genres','IMDb')
plt.xlabel('Genres')
plt.ylabel('IMDb rating')
plt.show()

In [ ]:
# The next example will compute the Pearson correlation coefficient between "Genres" and "IMDb" for Netflix streaming
#df_NetflixGenre.fillna(0, inplace=True, downcast='infer')
#df_NetflixGenre['Genres']=df_NetflixGenre['Genres'].astype('category')
#df_NetflixGenre['Rotten Tomatoes']=df_NetflixGenre['Rotten Tomatoes'].astype('category')
#.cat.codes
df_NetflixGenre.fillna(0, inplace=True, downcast='infer')
df_NetflixGenre.plot.scatter('Genres','Rotten Tomatoes')
plt.xlabel({"Genres":'Action', 'Adventure', 'Sci-Fi','Thriller', 'Comedy', 'Western',
       'Animation', 'Family', 'Biography', 'Drama', 'Music', 'War',
       'Crime', 'Fantasy', 'Romance', 'History', 'Mystery', 'Horror',
       'Sport', 'Documentary', 'Musical', 'News', 'Short', 'Reality-TV',
       'Talk-Show', 'Game-Show', 'Film-Noir'})
plt.ylabel('Rotten Tomatoes')
plt.show()

In [ ]:
df_Hulu=Streaming_drop_df[Streaming_drop_df['Hulu']==1]
len(df_Hulu)

In [ ]:
df_Hulu.Genres.mode()

In [ ]:
df_Prime_Video=Streaming_drop_df[Streaming_drop_df['Prime Video']==1]
len(df_Prime_Video)

In [ ]:
df_Prime_Video.Genres.mode()

In [ ]:
df_Disneyp=Streaming_drop_df[Streaming_drop_df['Disney+']==1]
len(df_Disneyp)

In [ ]:
df_Disneyp.Genres.mode()

In [ ]:
strm_split_Genres.Netflix.sum()

In [ ]:
strm_split_Genres.Hulu.sum()

In [ ]:
#description
strm_split_Genres.describe()

In [ ]:
#streaming_file_df.describe()

In [ ]:
#strm_split_Genres.hist(color='Blue', figsize=(10,10));

In [ ]:
#grp by genre and get mean per genre,drop year column for  mean calc

genremean=strm_split_Genres.groupby(['Genres']).mean()
genremean1=genremean.drop(['Year'],axis=1)
genremean2=genremean1.sort_values(by='IMDb', ascending=False)
genremean2.head()

In [ ]:
genremean2.describe()

In [ ]:
#grp by genre and get mean per genre,drop year column for  mean calc

genremean=strm_split_Genres.groupby(['Genres']).mean()
genremean1=genremean.drop(['Year'],axis=1)
genremean3=genremean1.sort_values(by='Rotten Tomatoes', ascending=False)
genremean3.head()

In [ ]:
strm_split_Genres2=strm_split_Genres.drop_duplicates()
strm_split_Genres2_df=pd.DataFrame(strm_split_Genres2)
strm_split_Genres2.nlargest(6,'Rotten Tomatoes', keep='first')

In [ ]:
strm_split_Genres2=strm_split_Genres.drop_duplicates()
strm_split_Genres2.nlargest(6,'IMDb', keep='first')

In [ ]:
strm_split_Genres.Genres.unique()

In [ ]:
Genrecount_df=pd.DataFrame({'count' : strm_split_Genres2.groupby( [ "Genres"] ).size()}).reset_index()
Genrecount_df.head(20)
    
    

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
Genrecount_df.plot(x='Genres', y='count', kind='bar', legend=False)
plt.show()

In [ ]:
#Genrecount_df.plot(x='Genres', y='count', kind='bar', legend=False)
#GenrecountN=pd.DataFrame({'count': df_NetflixGenre.groupby(["Genres"] ).size().reset_index()})
#errors='ignore'
#GenrecountN.head()

In [ ]:
#Netflix by genre
import pandas as pd
import matplotlib.pyplot as plt
GenrecountN_df.plot(x='Genres', y='count', kind='bar', legend='True')
plt.show

In [ ]:
# use a visualization to display that same data, including a title and x and y labels
strm_split_Genres[['IMDb','Genres']].groupby('IMDb').count().plot(kind='bar', title='Content Rating Visualization')
plt.xlabel('Content Rating')
plt.ylabel('Title Count')

In [ ]:
# determine the top rated movie (by star rating) for each genre
strm_data.sort_values('Rotten Tomatoes', ascending=False).groupby('Genres')['Title','Rotten Tomatoes'].first()

In [ ]:
#correlation bet mean IMDB and Rotten Tomatoes by Genre
strm_split_Genres
plt.scatter(genremean3.iloc[:,3],genremean3.iloc[:,4])
plt.xlabel('IMDb')
plt.ylabel('Rotten Tomatoes')
plt.show()
correlation = st.pearsonr('IMDb','Rotten Tomatoes')
#numpy.corrcoef('IMDb','Rotten Tomatoes')[0, 1]
print(f"The correlation between both factors is {round(correlation[0],2)}")

In [ ]:
#correlation bet mean IMDB and Rotten Tomatoes by Genre
#strm_split_Genres
plt.scatter(streaming_file_df.iloc[:,4],streaming_file_df.iloc[:12])
plt.xlabel('IMDb')
plt.ylabel('Runtime')
plt.show()
correlation = st.pearsonr('IMDb','Runtime')
#numpy.corrcoef('IMDb','Rotten Tomatoes')[0, 1]
print(f"The correlation between both factors is {round(correlation[0],2)}")

In [29]:
# determine the top rated movie (by star rating) for each genre
strm_split_Genres.sort_values('IMDb', ascending=False).groupby('Genres')['Title'].first()

Genres
Action                                           The Dark Knight
Adventure                                                 Bounty
Animation                                           Memories 677
Biography                                      A Dog Named Gucci
Comedy                                           Love on a Leash
Crime                                            The Dark Knight
Documentary                                   Down, But Not Out!
Drama                                                 Square One
Family                                            Finding Family
Fantasy                                          Love on a Leash
Film-Noir                                       Sunset Boulevard
Game-Show                                    5 Seconds of Summer
History                                                   Bounty
Horror                                              Memories 677
Music                                                 Square One
Musical           

In [30]:
# determine the top rated movie (by star rating) for each genre
strm_split_Genres.sort_values('Rotten Tomatoes', ascending=False).groupby('Genres')['Title'].first()

Genres
Action                              A Trip to the Moon
Adventure                                 Captain Kidd
Animation                     Mickey's Christmas Carol
Biography                                 Captain Kidd
Comedy                       The Distinguished Citizen
Crime                          Abducted in Plain Sight
Documentary    To Hell and Back: The Kane Hodder Story
Drama                        The Distinguished Citizen
Family                                    Mary Poppins
Fantasy                                   Mary Poppins
Film-Noir                                     Gaslight
Game-Show                                  Double Dare
History                                   Captain Kidd
Horror               Nightmares in Red, White and Blue
Music                           Sun Ra: A Joyful Noise
Musical                                   Mary Poppins
Mystery                                       Gaslight
News                             Life According to Sam
Rea

In [36]:
# calculate the average star rating for each genre, but only include genres with at least 10 movies
genres = strm_split_Genres[[Genres].count>10].index()
strm_split_Genres[strm_split_Genres['Genres'].isin('Genres')].groupby('Genres')['IMDb'].mean()

NameError: name 'Genres' is not defined